# Sistema RAG con LangChain y Pinecone

Este notebook implementa un sistema de **Retrieval-Augmented Generation (RAG)** que permite realizar consultas inteligentes sobre documentos web utilizando:
- **LangChain**: Framework para aplicaciones con LLMs
- **OpenAI GPT-4**: Modelo de lenguaje para generación de respuestas
- **Pinecone**: Base de datos vectorial para búsqueda semántica
- **BeautifulSoup4**: Parsing de contenido HTML

---

## 1. Instalación de Dependencias

En esta sección instalamos todas las librerías necesarias para el proyecto:
- **openai**: Cliente de OpenAI para acceso a GPT-4 y embeddings
- **python-dotenv**: Gestión de variables de entorno
- **langchain**: Framework principal para RAG
- **langchain-text-splitters**: Herramientas para dividir documentos
- **langchain-community**: Loaders de documentos (WebBaseLoader)
- **bs4 (BeautifulSoup)**: Parser HTML
- **langchain-openai**: Integración específica de LangChain con OpenAI
- **langchain-pinecone**: Integración con Pinecone Vector Database

In [ ]:
%pip install openai python-dotenv

In [ ]:
%pip install langchain langchain-text-splitters langchain-community bs4

In [ ]:
%pip install -U "langchain[openai]"

In [ ]:
%pip install -U "langchain-openai"

In [ ]:
%pip install -qU langchain-pinecone

## 2. Importación de Librerías

Importamos todas las dependencias necesarias organizadas por funcionalidad:
1. **Variables de entorno y cliente OpenAI**
2. **Modelos de chat y embeddings**
3. **Pinecone vector store**
4. **Web scraping y document loaders**
5. **Text splitters para chunking**
6. **Tools y agents de LangChain**

In [34]:
# Cargar las dependencias de env
import os
from dotenv import load_dotenv
from openai import OpenAI

In [35]:
from langchain.chat_models import init_chat_model
from langchain_openai import OpenAIEmbeddings

In [36]:
from langchain_pinecone import PineconeVectorStore
from pinecone import Pinecone

In [37]:
import bs4
from langchain_community.document_loaders import WebBaseLoader

In [38]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [39]:
from langchain.tools import tool
from langchain.agents import create_agent

## 3. Configuración Inicial

### 3.1 Variables de Entorno y Cliente OpenAI
Cargamos las credenciales desde el archivo `.env`:
- `OPENAI_API_KEY`: Para acceso a GPT-4 y embeddings
- `PINECONE_API_KEY`: Para acceso a Pinecone
- `PINECONE_INDEX_NAME`: Nombre del índice vectorial
- `LANGCHAIN_API_KEY`: Para tracing (opcional)
- `LANGCHAIN_TRACING`: Activar monitoreo

In [40]:
load_dotenv()  # Lee el archivo .env
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
langchain_api_key = os.getenv("LANGCHAIN_API_KEY")
langchain_tracing = os.getenv("LANGCHAIN_TRACING")


print("Cliente inicializado. Modelo listo para consultas.")

Cliente inicializado. Modelo listo para consultas.


### 3.2 Inicialización de Modelos
Configuramos los modelos de IA:
- **GPT-4.1**: Modelo de lenguaje para generación de respuestas
- **text-embedding-3-large**: Modelo de embeddings con 1024 dimensiones para vectorización de texto

In [41]:
model = init_chat_model("gpt-4.1")
embeddings = OpenAIEmbeddings(model="text-embedding-3-large",dimensions=1024)

### 3.3 Configuración de Pinecone Vector Store
Conectamos con Pinecone y creamos el vector store:
- Inicializamos el cliente de Pinecone con la API key
- Conectamos al índice existente
- Creamos el PineconeVectorStore que usará los embeddings de OpenAI para indexar y buscar documentos

In [42]:
pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))
index = pc.Index(os.getenv("PINECONE_INDEX_NAME"))

vector_store = PineconeVectorStore(embedding=embeddings, index=index)

## 4. Carga y Procesamiento de Documentos

### 4.1 Carga de Documento Web
Utilizamos `WebBaseLoader` para extraer contenido de un blog post:
- **URL**: Blog de Lilian Weng sobre agentes de IA
- **Parser**: BeautifulSoup con filtro para extraer solo título, headers y contenido
- **Resultado**: Un documento con todo el texto relevante del post

In [43]:
# Only keep post title, headers, and content from the full HTML.
bs4_strainer = bs4.SoupStrainer(class_=("post-title", "post-header", "post-content"))
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs={"parse_only": bs4_strainer},
)
docs = loader.load()

assert len(docs) == 1
print(f"Total characters: {len(docs[0].page_content)}")

Total characters: 43047


### 4.2 Visualización del Contenido
Mostramos los primeros 500 caracteres del documento cargado para verificar que se extrajo correctamente el contenido.

In [44]:
print(docs[0].page_content[:500])



      LLM Powered Autonomous Agents
    
Date: June 23, 2023  |  Estimated Reading Time: 31 min  |  Author: Lilian Weng


Building agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be framed as a powerful general problem solver.
Agent System Overview#
In


### 4.3 División de Documentos en Chunks
Utilizamos `RecursiveCharacterTextSplitter` para dividir el documento en fragmentos manejables:
- **Chunk Size**: 1000 caracteres por fragmento
- **Chunk Overlap**: 200 caracteres de superposición entre chunks para mantener contexto
- **add_start_index**: Rastrea la posición original en el documento
- **Propósito**: Fragmentos pequeños mejoran la precisión de la búsqueda semántica

In [45]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,  # chunk size (characters)
    chunk_overlap=200,  # chunk overlap (characters)
    add_start_index=True,  # track index in original document
)
all_splits = text_splitter.split_documents(docs)

print(f"Split blog post into {len(all_splits)} sub-documents.")

Split blog post into 63 sub-documents.


### 4.4 Indexación en Pinecone
Agregamos los documentos fragmentados al vector store:
- Cada chunk se convierte en un embedding de 1024 dimensiones
- Los embeddings se almacenan en Pinecone junto con los metadatos
- Retorna los IDs de los documentos indexados
- **Nota**: `add_documents()` puede retornar `None` en algunas versiones

In [46]:
document_ids = vector_store.add_documents(documents=all_splits)

print(document_ids[:3])

['2e7d2b65-7738-4fe2-a041-e526140778a7', 'bc6bf7d7-8c0f-4914-9a17-f1c656482adc', 'e40d98b6-ee17-4889-931c-a6e4ac4d7998']


## 5. Creación del Agente RAG

### 5.1 Definición de la Herramienta de Recuperación
Creamos una herramienta personalizada que:
- Recibe una query del usuario
- Realiza búsqueda de similitud en Pinecone (top k=2)
- Retorna tanto el contenido serializado como los documentos originales
- Usa `response_format="content_and_artifact"` para devolver ambos formatos

In [47]:
@tool(response_format="content_and_artifact")
def retrieve_context(query: str):
    """Retrieve information to help answer a query."""
    retrieved_docs = vector_store.similarity_search(query, k=2)
    serialized = "\n\n".join(
        (f"Source: {doc.metadata}\nContent: {doc.page_content}")
        for doc in retrieved_docs
    )
    return serialized, retrieved_docs

### 5.2 Configuración del Agente
Creamos el agente con:
- **Modelo**: GPT-4.1
- **Tools**: La función `retrieve_context` definida anteriormente
- **System Prompt**: Instrucciones para que el agente use la herramienta de recuperación al responder consultas

In [48]:
tools = [retrieve_context]
# If desired, specify custom instructions
prompt = (
    "You have access to a tool that retrieves context from a blog post. "
    "Use the tool to help answer user queries."
)
agent = create_agent(model, tools, system_prompt=prompt)

## 6. Ejecución de Consultas

### 6.1 Consulta de Ejemplo
Ejecutamos una consulta compleja que requiere:
1. Buscar información sobre "Task Decomposition"
2. Una vez obtenida la respuesta, buscar extensiones de ese método
3. El agente decide cuándo usar la herramienta de recuperación
4. Combina la información recuperada para generar una respuesta coherente

**Modo de streaming**: Las respuestas se muestran en tiempo real a medida que se generan.

In [49]:
query = (
    "What is the standard method for Task Decomposition?\n\n"
    "Once you get the answer, look up common extensions of that method."
)

for event in agent.stream(
    {"messages": [{"role": "user", "content": query}]},
    stream_mode="values",
):
    event["messages"][-1].pretty_print()

================================ Human Message =================================

What is the standard method for Task Decomposition?

Once you get the answer, look up common extensions of that method.
================================== Ai Message ==================================
Tool Calls:
  retrieve_context (call_I5d25JSTu1QXuulxcB9Cu0d5)
 Call ID: call_I5d25JSTu1QXuulxcB9Cu0d5
  Args:
    query: standard method for Task Decomposition
================================== Ai Message ==================================
Tool Calls:
  retrieve_context (call_I5d25JSTu1QXuulxcB9Cu0d5)
 Call ID: call_I5d25JSTu1QXuulxcB9Cu0d5
  Args:
    query: standard method for Task Decomposition
================================= Tool Message =================================
Name: retrieve_context

Source: {'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/', 'start_index': 2578.0}
Content: Task decomposition can be done (1) by LLM with simple prompting like "Steps for XYZ.\n1.", "What are t